# Transformaciones precios
En este notebook se harán las transformaciones que se discuten en el notebook de exploración de datasets sobre los dataset de precios.

In [1]:
import pandas as pd
import unidecode
import numpy as np

### precios_semana_20200413.csv

In [2]:
precios_0413 = pd.read_csv('../datasets/precios_semana_20200413.csv')
precios_0413.head()

,precio,producto_id,sucursal_id
0,29.90,0000000001663,2-1-014
1,29.90,0000000002288,2-1-032
2,39.90,0000000002288,2-1-096
3,499.99,0000000205870,9-1-686
4,519.99,0000000205870,9-2-248


In [3]:
# hacemos tabla auxiliar para los nulos
precios_aux = precios_0413.loc[(precios_0413.producto_id.isnull()) | (precios_0413.sucursal_id.isnull())].copy()
precios_aux['tipoError'] = 0 
precios_aux

,precio,producto_id,sucursal_id,tipoError
246393,NaN,NaN,NaN,0
246748,NaN,NaN,NaN,0
246882,NaN,NaN,NaN,0
246978,NaN,NaN,NaN,0
247033,NaN,NaN,NaN,0
247079,451.00,NaN,NaN,0
247167,NaN,NaN,NaN,0
247310,NaN,NaN,NaN,0
247404,NaN,NaN,NaN,0
247405,NaN,NaN,NaN,0


In [4]:
# Dropeamos nulos 
index_nulls = precios_0413.loc[(precios_0413.producto_id.isnull()) | (precios_0413.sucursal_id.isnull())].index.to_list()
precios_0413.drop(index_nulls, axis='index', inplace=True)
# revisamos que no hayan nulos
precios_0413.loc[(precios_0413.producto_id.isnull()) | (precios_0413.sucursal_id.isnull())]

,precio,producto_id,sucursal_id


In [5]:
# hay productos con los productos id de más de 13 digitos
precios_0413.loc[precios_0413.producto_id.str.len() != 13]

,precio,producto_id,sucursal_id
11540,59.00,10-1-2300073000005,10-1-25
11541,7.49,10-1-2300073000005,10-1-48
11542,59.00,10-1-2300075000003,10-1-25
11543,279.90,10-1-2300160000000,10-1-8
11544,1274.00,10-1-2300175000002,10-1-6
...,...,...,...
472008,387.00,9-3-0000000994002,9-3-5222
472009,509.00,9-3-0000000995696,9-3-628
472010,90.99,9-3-0000000997096,9-3-5222
472011,219.00,9-3-0000000997126,9-3-5222


In [6]:
# por ahora parece estar bien
precios_0413.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472151 entries, 0 to 472165
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   precio       472151 non-null  float64
 1   producto_id  472151 non-null  object 
 2   sucursal_id  472151 non-null  object 
dtypes: float64(1), object(2)
memory usage: 14.4+ MB


In [7]:
# le agregamos el idSemana
filename = 'precios_semana_20200413.csv'
# Lo obtenemos del nombre del archivo
filename_split = filename.split('_')[2].split('.')
semana = filename_split[0]
tipo = filename_split[1]

In [8]:
precios_0413['semanaId'] = semana
precios_0413

,precio,producto_id,sucursal_id,semanaId
0,29.90,0000000001663,2-1-014,20200413
1,29.90,0000000002288,2-1-032,20200413
2,39.90,0000000002288,2-1-096,20200413
3,499.99,0000000205870,9-1-686,20200413
4,519.99,0000000205870,9-2-248,20200413
...,...,...,...,...
472161,139.99,9569753142128,25-1-1,20200413
472162,34.99,9795403001143,25-1-1,20200413
472163,312.50,9990385651922,5-1-3,20200413
472164,312.50,9990385651939,5-1-3,20200413


In [9]:
# Agrego id y reordeno
precios_0413.reset_index(drop=True, inplace=True)
precios_0413['precioId'] = precios_0413.index + 1
precios_0413 = precios_0413[['precioId','precio','producto_id','sucursal_id','semanaId']].copy()
precios_0413.rename({'producto_id':'productoId','sucursal_id':'sucursalId'}, axis='columns', inplace=True)
precios_0413

,precioId,precio,productoId,sucursalId,semanaId
0,1,29.90,0000000001663,2-1-014,20200413
1,2,29.90,0000000002288,2-1-032,20200413
2,3,39.90,0000000002288,2-1-096,20200413
3,4,499.99,0000000205870,9-1-686,20200413
4,5,519.99,0000000205870,9-2-248,20200413
...,...,...,...,...,...
472146,472147,139.99,9569753142128,25-1-1,20200413
472147,472148,34.99,9795403001143,25-1-1,20200413
472148,472149,312.50,9990385651922,5-1-3,20200413
472149,472150,312.50,9990385651939,5-1-3,20200413


In [10]:
# cambio los indices de producto por los nuevos
prod_aux = pd.read_csv('../datasets/producto_auxiliar.csv') #se genera en transformaciones_producto.ipynb
precios_0413 = pd.merge(precios_0413, prod_aux, left_on = 'productoId', right_on = 'antiguoId', how = 'left')
precios_0413.drop(['productoId_x','antiguoId'], axis=1, inplace=True)
precios_0413.drop('antiguoId', axis=1, inplace=True)
precios_0413.rename({'productoId_y':'productoId'}, axis='columns', inplace=True)
precios_0413 = precios_0413[['precioId','precio','productoId','sucursalId','semanaId']].copy()
precios_0413

KeyError: "['antiguoId'] not found in axis"

In [ ]:
# Guardo a un csv
precios_0413.to_csv('../datasets/precios_0413.csv', index=False)

### precios_semanas_20200419_20200426.xlsx

In [ ]:
xls = pd.ExcelFile('../datasets/precios_semanas_20200419_20200426.xlsx')
precios_0426 = pd.read_excel(xls, 'precios_20200426_20200426')
precios_0419 = pd.read_excel(xls, 'precios_20200419_20200419')

In [ ]:
filename = 'precios_semanas_20200419_20200426.xlsx'

semana_0419 = filename.split('_')[2]
filename_split = filename.split('_')[3].split('.')
semana_0426 = filename_split[0]
tipo = filename_split[1]

In [ ]:
precios_0419

In [ ]:
type(precios_0419.producto_id[0])

In [ ]:
# Arreglamos el producto_id para que sea un codigo EAN
precios_0419.producto_id = precios_0419.producto_id.astype(str).str.replace('.0', '', regex=False)
precios_0419.producto_id = precios_0419.producto_id.str.zfill(13)
precios_0419.head()

In [ ]:
'''
# Hay timestamps en sucursal_id
precios_0419.sucursal_id = precios_0419.sucursal_id.astype(str)
#precios_0419.sucursal_id.apply(lambda col: col.find('00:00:00'))
# Hay muchas sucursales con timestamp 
precios_0419.loc[precios_0419.sucursal_id.str.split(expand=True)[1].notnull(),'sucursal_id'] = 'Sin dato'
'''

In [ ]:
# le agregamos el codigo de semana 
precios_0419['semanaId'] = semana_0419
precios_0419

In [ ]:
precios_0419.info()

### precios_semana_20200503.json

In [ ]:
precios_0503 = pd.read_json('../datasets/precios_semana_20200503.json')
precios_0503.head()

In [ ]:
precios_0503.info()

In [ ]:
precios_0503.producto_id = precios_0503.producto_id.astype(str)

In [ ]:
# revisamos los idproducto
precios_0503.loc[precios_0503.producto_id.str.len() != 13]

In [ ]:
# le agregamos el idSemana
filename = 'precios_semana_20200503.json'
# Lo obtenemos del nombre del archivo
filename_split = filename.split('_')[2].split('.')
semana = filename_split[0]
tipo = filename_split[1]

In [ ]:
# agregamos semanaId
precios_0503['semanaId'] = semana
precios_0503

### precios_semana_20200518.txt

In [ ]:
# no hay nulos en esta
precios_0518 = pd.read_csv('../datasets/precios_semana_20200518.txt', sep='|')
precios_0518.head()

In [ ]:
precios_0518.info()

In [ ]:
# elimino filas con nulos en sucursal y producto
index = precios_0518.loc[(precios_0518.producto_id.isnull()) & (precios_0518.sucursal_id.isnull())].index
precios_0518.drop(index, axis='index', inplace=True)

# reviso que no hay nulos
precios_0518.loc[(precios_0518.producto_id.isnull()) & (precios_0518.sucursal_id.isnull())]

In [ ]:
precios_0518.loc[precios_0518.precio.isnull()]

In [ ]:
# le agregamos el idSemana
filename = 'precios_semana_20200518.txt'
# Lo obtenemos del nombre del archivo
filename_split = filename.split('_')[2].split('.')
semana = filename_split[0]
tipo = filename_split[1]

In [ ]:
# agregamos semanaId
precios_0518['semanaId'] = semana
precios_0518

### Juntamos precios

### Tratamiento de nulos
Las tablas con nulos son 0419, 0426 y 0518

In [ ]:
# pero el porcentaje de nulos es muy bajo
round((precios_0419.loc[precios_0419.precio.isnull()].shape[0]/precios_0419.shape[0])*100,2)

In [ ]:
# Tabla auxiliar con precios nulos
precios_aux = precios_0419.loc[precios_0419.precio.isnull()].copy()
precios_aux['tipoError'] = 0
precios_aux.head()

In [ ]:
# los relleno con la moda
precios_0419.loc[precios_0419.precio.isnull(),'precio'] = precios_0419.precio.mode()[0]
precios_0419.loc[precios_0419.precio.isnull()]

In [ ]:
# agregamos los que tienen precios faltantes al aux
df = precios_0426.loc[precios_0426.precio.isnull()].copy()
df['tipoError'] = 0
precios_aux = pd.concat([precios_aux, df])
del df
precios_aux.head()

In [ ]:
# pero el porcentaje de nulos es muy bajo
round((precios_0426.loc[precios_0426.precio.isnull()].shape[0]/precios_0426.shape[0])*100,2)

In [ ]:
# los relleno con la moda
precios_0426.loc[precios_0426.precio.isnull(),'precio'] = precios_0426.precio.mode()[0]
precios_0426.loc[precios_0426.precio.isnull()]

In [ ]:
(precios_0518.loc[precios_0518.precio.isnull()].shape[0]/precios_0518.shape[0])*100